In [24]:
import torch
from torch import nn
from skopt import gp_minimize, dump, load
from skopt.callbacks import DeltaYStopper
from scipy.stats.qmc import LatinHypercube
from pathlib import Path

from mlp import MLP, train_mlp
from objective import objective, search_space
from duqling_py import Duqling

In [18]:
duqling = Duqling()

In [19]:
def lhs_array(n: int, d: int, ranges: torch.Tensor | None = None, seed: int | None = None) -> torch.Tensor:
    sampler = LatinHypercube(d, seed=seed)
    samples = sampler.random(n)
    if ranges is not None:
        samples = ranges[:, 0] + samples * (ranges[:, 1] - ranges[:, 0])
    return torch.FloatTensor(samples)

In [20]:
fname = 'banana'

func_info = duqling.quack(fname)
input_dim = func_info['input_dim']

X_train = lhs_array(1000, input_dim)
X_val   = lhs_array(1000, input_dim)
X_test  = lhs_array(1000, input_dim)

y_train = duqling.duq(X_train, fname).reshape(-1, 1)
y_val   = duqling.duq(X_val,   fname).reshape(-1, 1)
y_test  = duqling.duq(X_test,  fname).reshape(-1, 1)

delta_stopper = DeltaYStopper(delta=1e-4)
result = gp_minimize(
    func=lambda params: objective(params, X_train, y_train, X_val, y_val),
    dimensions=search_space,
    n_calls=30,
    random_state=42,
    acq_func='EI',
    callback=delta_stopper,
    verbose=True
)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.9771
Function value obtained: 132.3322
Current minimum: 132.3322
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.5992
Function value obtained: 144699.3023
Current minimum: 132.3322
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 2.0763
Function value obtained: 159009.3615
Current minimum: 132.3322
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.8922
Function value obtained: 3090.8333
Current minimum: 132.3322
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.6304
Function value obtained: 224060.1153
Current minimum: 132.3322
Itera

In [38]:
depth, width, lr, activation = result.x
model = MLP(input_dim, [width] * depth, activation)
train_mlp(model, X_train, y_train, X_val, y_val, lr)
mse = nn.MSELoss()
print(f"Test RMSE: {mse(model(X_test), y_test):.4f}")

Test RMSE: 97.9603
